In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../data/guland_hanoi_listings_arcgis1.csv")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(df.isna().sum())
print("Dups: ",df.duplicated().sum())

In [ ]:
df[df.duplicated()].head(10)

In [ ]:
df.info()

## Transform the column Prize and Area to float / number type

- The common case is that "ty" often refer to the price of the whole estate, while "trieu" refer to the price of an square meter, but that is not the true case all the time, for real estate that have small area, [some number] + "trieu" could refer to the prize of the whole avenue. How to deal with this?
- Fuck the above point also depend on the location? Like a venue in Soc Son, with 151m2, this should have really high prize if in inner districts of Hanoi, but here only 750 "trieu" for the whole area


- Remove all nan in Price and Area

In [ ]:
df = df[df['Price'].notna() & df['Area'].notna()]

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

In [ ]:
print(df.isna().sum())
print("Dups: ",df.duplicated().sum())

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.info()

- First remove rows that have no "trieu" or "ty" in 'Prize' columns, we dont want to deal with that and classify manually

In [ ]:
df[~(df['Price'].str.contains("tỷ") | df['Price'].str.contains("triệu"))].info()


- Check the number of rows that have "trieu"

In [ ]:
df['Price'].str.contains("triệu").value_counts()

- 252? we decide to delete those cols since we have >20k rows amt, might revisit later if we can come up with another way

In [ ]:
df = df[~df['Price'].str.contains("triệu", na = False)]
df.info()

In [ ]:
global count_invalid
count_invalid = 0  # Initialize the global variable

def sanitize_and_cv_to_float(price) -> float:
    global count_invalid
    price = str(price)
    if "tỷ" in price:
        price = price.replace("tỷ", "").strip()
    if "m²" in price:
        price = price.replace("m²", "").strip()
    try:
        return float(price)
    except:
        count_invalid += 1
        return np.nan
    


In [ ]:
df['Price'] = np.vectorize(sanitize_and_cv_to_float)(df['Price'])
print(count_invalid)

In [ ]:
count_invalid = 0
df['Area'] = np.vectorize(sanitize_and_cv_to_float)(df['Area'])
print(count_invalid)
df['Price per m2'] = 1e3 * df['Price'] / df['Area']

In [ ]:
df.head(10)

In [ ]:
df.isna().sum()

## Remove Outliers

In [ ]:
num_cols_name = ['Price', 'Area', 'Longitude', 'Latitude', 'Price per m2']
# num_cols_name = ['Longitude', 'Latitude']

# num_cols_name = ['Longitude', 'Latitude']


plt.figure(figsize=(15, 10))
for i, col in enumerate(num_cols_name):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(x=df[col])
    plt.title(f'Distribution of {col}')

In [ ]:
Q1 = df[num_cols_name].quantile(0.25)
Q3 = df[num_cols_name].quantile(0.75)
IQR = Q3 - Q1

# df_cleaned = df[(df[num_cols_name] >= Q1 - 1.5 * IQR) & (df[num_cols_name] <= Q3 + 1.5 * IQR).all(axis=1)]
# df_cleaned.info()
# TOOO SLOWWW

conditions = []
for col in num_cols_name:
    lower = Q1[col] - 1.5 * IQR[col]
    upper = Q3[col] + 1.5 * IQR[col]
    conditions.append(f"({lower} <= `{col}` <= {upper})")

query_string = " and ".join(conditions)
df = df.query(query_string)
df.reset_index(drop=True, inplace=True)
df.info()

In [ ]:
print(df.duplicated().sum())
print(df.duplicated(['Price', 'Area', 'Latitude', 'Longitude']).sum())

In [ ]:
print(df.duplicated(['Latitude', 'Longitude']).sum())

In [ ]:
df.info()

In [ ]:
df.to_csv("../data/guland_hanoi_listings_arcgis2.csv", index=False)